# Virtual Fields Method: Material Parameter Identification

## Learning Objectives
In this exercise, you will:
1. Understand the VFM methodology for identifying material stiffness parameters
2. Load and explore experimental strain field data
3. Analyze VFM results and sensitivity parameters
4. Perform mesh convergence studies
5. Visualize strain and virtual fields

## Time: 45 minutes

## Background
The Virtual Fields Method (VFM) identifies material parameters from full-field strain measurements.
For an orthotropic material, we identify the stiffness matrix components:

$$Q_{11}, Q_{22}, Q_{12}, Q_{66}$$

where:
- $Q_{11}$: Longitudinal stiffness
- $Q_{22}$: Transverse stiffness
- $Q_{12}$: Coupling stiffness
- $Q_{66}$: Shear stiffness

## Prerequisites
- CSV files: `scalarsFE.csv` and `FEM2VFM.csv`
- Understanding of elasticity and strain tensors

---

## Part 1: Setup and Imports (Provided)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from scipy import io
from io import StringIO
from matplotlib.ticker import ScalarFormatter

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Part 2: VFM Class Structure (Provided)

The class structure and core VFM algorithm are provided. Review but don't modify.

In [2]:
class VFMPiecewise:
    """
    Virtual Fields Method implementation using optimized special piecewise virtual fields
    
    Students will complete key analysis and visualization methods.
    """
    
    def __init__(self, data_source='csv'):
        self.data_source = data_source
        self.data = {}
        self.results = {}
    
    def load_data_from_csv(self, csv_dir='.'):
        """Load data from CSV files (PROVIDED)"""
        # Load scalar parameters
        scalar_path = os.path.join(csv_dir, 'scalarsFE.csv')
        
        if os.path.exists(scalar_path):
            with open(scalar_path, 'r') as f:
                for line in f:
                    if '=' in line:
                        key, value = line.strip().split('=')
                        key = key.strip()
                        value = float(value.strip())
                        
                        if key == 'Length':
                            self.data['L'] = value
                        elif key == 'Width':
                            self.data['w'] = value
                        elif key == 'Thick':
                            self.data['t'] = value
                        elif key == 'P':
                            self.data['F'] = value
        
        # Load FEM data
        fem_path = os.path.join(csv_dir, 'FEM2VFM.csv')
        
        if os.path.exists(fem_path):
            fem_data = pd.read_csv(fem_path, sep=r'\s+')
            
            self.data['X1'] = fem_data['X_Coord'].values
            self.data['X2'] = fem_data['Y_Coord'].values
            self.data['Eps1'] = fem_data['Eps_X'].values
            self.data['Eps2'] = fem_data['Eps_Y'].values
            self.data['Eps6'] = fem_data['Eps_XY'].values
            
            # Grid dimensions
            unique_x1 = np.unique(self.data['X1'])
            unique_x2 = np.unique(self.data['X2'])
            n_column = len(unique_x1)
            n_row = len(unique_x2)
            
            self.data['n_row'] = n_row
            self.data['n_column'] = n_column
            
            # Reshape to 2D
            try:
                self.data['X1_2D'] = self.data['X1'].reshape(n_row, n_column)
                self.data['X2_2D'] = self.data['X2'].reshape(n_row, n_column)
                self.data['Eps1_2D'] = self.data['Eps1'].reshape(n_row, n_column)
                self.data['Eps2_2D'] = self.data['Eps2'].reshape(n_row, n_column)
                self.data['Eps6_2D'] = self.data['Eps6'].reshape(n_row, n_column)
            except ValueError:
                print(f"Warning: Could not reshape to {n_row}x{n_column} grid")
        
        print("Data loaded successfully!")

## Exercise 1: Data Exploration (10 minutes)

Complete the data validation function to understand the loaded experimental data.

In [3]:
def display_data_summary(self):
    """
    Display a comprehensive summary of loaded data
    
    Students: Complete this function to explore the data
    """
    
    print("="*70)
    print("DATA SUMMARY")
    print("="*70)
    
    # TODO: Print specimen geometry
    # Hint: Access self.data['L'], self.data['w'], self.data['t']
    print("\nSpecimen Geometry:")
    print(f"  Length (L): {# YOUR CODE HERE} mm")
    print(f"  Width (w): {# YOUR CODE HERE} mm")
    print(f"  Thickness (t): {# YOUR CODE HERE} mm")
    
    # TODO: Print applied force
    print(f"  Applied Force (F): {# YOUR CODE HERE} N")
    
    # TODO: Print grid information
    print("\nMeasurement Grid:")
    print(f"  Grid size: {# YOUR CODE HERE} × {# YOUR CODE HERE} points")
    print(f"  Total data points: {# YOUR CODE HERE}")
    
    # TODO: Print strain field statistics
    # Hint: Use np.min(), np.max(), np.mean(), np.std()
    print("\nStrain Field Statistics:")
    
    Eps1 = self.data['Eps1']
    print(f"  ε₁: min={# YOUR CODE HERE:.2e}, max={# YOUR CODE HERE:.2e}, mean={# YOUR CODE HERE:.2e}")
    
    Eps2 = self.data['Eps2']
    print(f"  ε₂: min={# YOUR CODE HERE:.2e}, max={# YOUR CODE HERE:.2e}, mean={# YOUR CODE HERE:.2e}")
    
    Eps6 = self.data['Eps6']
    print(f"  ε₆: min={# YOUR CODE HERE:.2e}, max={# YOUR CODE HERE:.2e}, mean={# YOUR CODE HERE:.2e}")
    
    print("="*70)

# Add method to class
VFMPiecewise.display_data_summary = display_data_summary

SyntaxError: f-string expression part cannot include '#' (3784437595.py, line 15)

## Part 3: Core VFM Algorithm (Provided - Complete)

This is the heart of the VFM. It's complex, so we provide it complete.
Review the algorithm structure but focus on understanding the inputs and outputs.

In [ ]:
def vfm_piecewise_function(self, m=4, n=5):
    """
    Main VFM piecewise function - PROVIDED COMPLETE
    
    Parameters:
    -----------
    m : int - Number of elements along x1 (horizontal)
    n : int - Number of elements along x2 (vertical)
    
    Returns:
    --------
    Q : array [Q11, Q22, Q12, Q66] - Material stiffness parameters (MPa)
    eta : array - Sensitivity parameters
    Ya, Yb, Yc, Yd : arrays - Virtual field coefficients
    """
    
    # Extract data
    Eps1 = self.data['Eps1']
    Eps2 = self.data['Eps2']
    Eps6 = self.data['Eps6']
    X1 = self.data['X1']
    X2 = self.data['X2']
    L = self.data['L']
    w = self.data['w']
    t = self.data['t']
    F = self.data['F']
    
    # Parameter definition
    n_nodes = (m + 1) * (n + 1)
    n_elem = m * n
    n_points = len(Eps1)
    
    n_row = self.data.get('n_row', int(np.sqrt(n_points)))
    n_column = self.data.get('n_column', int(np.sqrt(n_points)))
    
    L_el = L / m
    w_el = w / n
    
    # Adjust coordinates
    X2_adjusted = X2 - np.min(X2) + w / n_row / 2
    
    X1_vec = X1
    X2_vec = X2_adjusted
    Eps1_vec = Eps1
    Eps2_vec = Eps2
    Eps6_vec = Eps6
    
    # Element indices
    iii = np.floor(X1_vec * m / L) + 1
    jjj = np.floor(X2_vec * n / w) + 1
    
    iii = np.clip(iii, 1, m).astype(int)
    jjj = np.clip(jjj, 1, n).astype(int)
    
    xsi1 = 2 * X1_vec / L_el - iii * 2 + 1
    xsi2 = 2 * X2_vec / w_el - jjj * 2 + 1
    
    # Virtual strain calculations
    Eps1elem = np.zeros((n_points, 8))
    Eps2elem = np.zeros((n_points, 8))
    Eps6elem = np.zeros((n_points, 8))
    u1elem = np.zeros((n_points, 8))
    u2elem = np.zeros((n_points, 8))
    
    for k in range(n_points):
        Eps1elem[k, :] = np.array([
            -(1 - xsi2[k]) / 2 / L_el, 0, (1 - xsi2[k]) / 2 / L_el, 0,
            (1 + xsi2[k]) / 2 / L_el, 0, -(1 + xsi2[k]) / 2 / L_el, 0
        ])
        
        Eps2elem[k, :] = np.array([
            0, -(1 - xsi1[k]) / 2 / w_el, 0, -(1 + xsi1[k]) / 2 / w_el,
            0, (1 + xsi1[k]) / 2 / w_el, 0, (1 - xsi1[k]) / 2 / w_el
        ])
        
        Eps6elem[k, :] = np.array([
            -(1 - xsi1[k]) / w_el / 2, -(1 - xsi2[k]) / L_el / 2,
            -(1 + xsi1[k]) / w_el / 2, (1 - xsi2[k]) / L_el / 2,
            (1 + xsi1[k]) / w_el / 2, (1 + xsi2[k]) / L_el / 2,
            (1 - xsi1[k]) / w_el / 2, -(1 + xsi2[k]) / L_el / 2
        ])
    
    # Matrix assembly
    B11 = np.zeros((1, 2 * n_nodes))
    B22 = np.zeros((1, 2 * n_nodes))
    B12 = np.zeros((1, 2 * n_nodes))
    B66 = np.zeros((1, 2 * n_nodes))
    
    H11 = np.zeros((2 * n_nodes, 2 * n_nodes))
    H22 = np.zeros((2 * n_nodes, 2 * n_nodes))
    H12 = np.zeros((2 * n_nodes, 2 * n_nodes))
    H66 = np.zeros((2 * n_nodes, 2 * n_nodes))
    
    n1 = (iii - 1) * (n + 1) + jjj
    n2 = iii * (n + 1) + jjj
    n3 = iii * (n + 1) + jjj + 1
    n4 = (iii - 1) * (n + 1) + jjj + 1
    
    assemble = np.column_stack([
        n1 * 2 - 1, n1 * 2, n2 * 2 - 1, n2 * 2,
        n3 * 2 - 1, n3 * 2, n4 * 2 - 1, n4 * 2
    ]).astype(int) - 1
    
    assemble = np.clip(assemble, 0, 2 * n_nodes - 1)
    
    # Assembly loop
    for k in range(n_points):
        assemble1 = assemble[k, :]
        
        B11[0, assemble1] += Eps1_vec[k] * Eps1elem[k, :] * L * w / n_points
        B22[0, assemble1] += Eps2_vec[k] * Eps2elem[k, :] * L * w / n_points
        B12[0, assemble1] += (Eps1_vec[k] * Eps2elem[k, :] + 
                              Eps2_vec[k] * Eps1elem[k, :]) * L * w / n_points
        B66[0, assemble1] += Eps6_vec[k] * Eps6elem[k, :] * L * w / n_points
        
        H11[np.ix_(assemble1, assemble1)] += np.outer(Eps1elem[k, :], Eps1elem[k, :])
        H22[np.ix_(assemble1, assemble1)] += np.outer(Eps2elem[k, :], Eps2elem[k, :])
        H12[np.ix_(assemble1, assemble1)] += np.outer(Eps1elem[k, :], Eps2elem[k, :])
        H66[np.ix_(assemble1, assemble1)] += np.outer(Eps6elem[k, :], Eps6elem[k, :])
    
    # Boundary conditions
    Aconst = np.zeros((4 * n + 3, 2 * n_nodes))
    
    for i in range(2 * (n + 1)):
        Aconst[i, i] = 1
    
    for i in range(n + 1):
        Aconst[i + 2 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * i] = 1
    
    for i in range(n):
        Aconst[i + 3 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * i + 1] = 1
        Aconst[i + 3 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * (i + 1) + 1] = -1
    
    # Z vectors
    Za = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
    Zb = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
    Zc = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
    Zd = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
    
    Za[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [1, 0, 0, 0]
    Zb[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 1, 0, 0]
    Zc[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 0, 1, 0]
    Zd[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 0, 0, 1]
    
    # Constraint matrix
    A = np.vstack([Aconst, B11, B22, B12, B66])
    B_zeros = np.zeros((A.shape[0], A.shape[0]))
    
    # Iterative optimization
    Q = np.array([1.0, 1.0, 1.0, 1.0])
    n_iter = 20
    delta_lim = 0.001
    delta = 10.0
    iteration = 1
    Q_old = Q.copy()
    
    while iteration < n_iter and delta > delta_lim:
        H = (L * w / n_points) ** 2 * (
            (Q[0] ** 2 + Q[2] ** 2) * H11 +
            (Q[1] ** 2 + Q[2] ** 2) * H22 +
            Q[3] ** 2 * H66 +
            2 * (Q[0] + Q[1]) * Q[2] * H12
        )
        
        corr = np.max(A) / np.max(H) if np.max(H) > 0 else 1.0
        
        OptM_top = np.hstack([H / 2 * corr, A.T * corr])
        OptM_bottom = np.hstack([A, B_zeros])
        OptM = np.vstack([OptM_top, OptM_bottom])
        
        try:
            Ya = np.linalg.solve(OptM, Za)
            Yb = np.linalg.solve(OptM, Zb)
            Yc = np.linalg.solve(OptM, Zc)
            Yd = np.linalg.solve(OptM, Zd)
            
            Ya = Ya[:2 * n_nodes]
            Yb = Yb[:2 * n_nodes]
            Yc = Yc[:2 * n_nodes]
            Yd = Yd[:2 * n_nodes]
            
            Q[0] = Ya[2 * n_nodes - 1] * F / t
            Q[1] = Yb[2 * n_nodes - 1] * F / t
            Q[2] = Yc[2 * n_nodes - 1] * F / t
            Q[3] = Yd[2 * n_nodes - 1] * F / t
            
            delta = np.sum((Q_old - Q) ** 2 / Q ** 2)
            
            iteration += 1
            Q_old = Q.copy()
            
        except np.linalg.LinAlgError as e:
            print(f"Error at iteration {iteration}: {e}")
            break
    
    # Final Hessian for sensitivity
    H_final = (L * w / n_points) ** 2 * (
        (Q[0] ** 2 + Q[2] ** 2) * H11 +
        (Q[1] ** 2 + Q[2] ** 2) * H22 +
        Q[3] ** 2 * H66 +
        2 * (Q[0] + Q[1]) * Q[2] * H12
    )
    
    # Sensitivity parameters
    eta = np.zeros(4)
    try:
        eta[0] = np.sqrt(Ya.T @ H_final @ Ya)
        eta[1] = np.sqrt(Yb.T @ H_final @ Yb)
        eta[2] = np.sqrt(Yc.T @ H_final @ Yc)
        eta[3] = np.sqrt(Yd.T @ H_final @ Yd)
    except:
        eta = np.zeros(4)
    
    return Q, eta, Ya, Yb, Yc, Yd

# Add method to class
VFMPiecewise.vfm_piecewise_function = vfm_piecewise_function

---
## STUDENT EXERCISE SECTION
---

### Exercise 2: Run VFM Analysis and Display Results (10 minutes)

**Task**: Complete the `run_analysis()` function to execute VFM and display results.

In [ ]:
def run_analysis(self, m=4, n=5):
    """
    Run VFM analysis with specified mesh density
    
    Parameters:
    -----------
    m : int - Number of elements horizontally (along x1)
    n : int - Number of elements vertically (along x2)
    """
    
    print("="*70)
    print("VFM PIECEWISE ANALYSIS")
    print("="*70)
    print(f"Mesh configuration: {m} × {n} elements")
    print(f"Total elements: {m * n}")
    print(f"Total nodes: {(m + 1) * (n + 1)}")
    print("\nRunning VFM optimization...")
    
    # TODO: Call the VFM function
    # Hint: Use self.vfm_piecewise_function(m, n)
    Q, eta, Ya, Yb, Yc, Yd = # YOUR CODE HERE
    
    # Convert to GPa for display
    Q_gpa = Q / 1e3
    
    # TODO: Display results
    print("\n" + "="*70)
    print("IDENTIFIED MATERIAL PARAMETERS")
    print("="*70)
    
    # TODO: Print each stiffness component with its sensitivity ratio
    # Format: Q11 = XXX.XX GPa    η/Q = X.XXXX
    # Hint: eta[i]/Q[i] gives the sensitivity ratio
    
    print(f"Q11 = {# YOUR CODE HERE:.2f} GPa\t\tη₁₁/Q₁₁ = {# YOUR CODE HERE:.4f}")
    print(f"Q22 = {# YOUR CODE HERE:.2f} GPa\t\tη₂₂/Q₂₂ = {# YOUR CODE HERE:.4f}")
    print(f"Q12 = {# YOUR CODE HERE:.2f} GPa\t\tη₁₂/Q₁₂ = {# YOUR CODE HERE:.4f}")
    print(f"Q66 = {# YOUR CODE HERE:.2f} GPa\t\tη₆₆/Q₆₆ = {# YOUR CODE HERE:.4f}")
    
    # Store results
    self.results = {
        'Q': Q,
        'eta': eta,
        'nodes': (m + 1) * (n + 1),
        'elements': m * n,
        'Ya': Ya, 'Yb': Yb, 'Yc': Yc, 'Yd': Yd,
        'm': m, 'n': n
    }
    
    return Q_gpa, eta

# Add method to class
VFMPiecewise.run_analysis = run_analysis

### Exercise 3: Compare with Reference Values (10 minutes)

**Theory**: We validate VFM results against known reference values (from literature or other methods).

**Task**: Complete the comparison function.

In [ ]:
def compare_with_reference(self, Q_ref, material_name="reference"):
    """
    Compare VFM results with reference values
    
    Parameters:
    -----------
    Q_ref : array - Reference values [Q11, Q22, Q12, Q66] in GPa
    material_name : str - Name of the material
    """
    
    if 'Q' not in self.results:
        print("No results available. Run analysis first.")
        return
    
    Q = self.results['Q']
    Q_gpa = Q / 1e3
    
    print("\n" + "="*70)
    print("COMPARISON WITH REFERENCE VALUES")
    print("="*70)
    print(f"Reference material: {material_name}")
    
    param_names = ['Q11', 'Q22', 'Q12', 'Q66']
    
    print(f"\n{'Parameter':<10} {'Computed':<12} {'Reference':<12} {'Error %':<10}")
    print("-"*70)
    
    # TODO: Calculate and display errors for each parameter
    for i, param in enumerate(param_names):
        # TODO: Calculate percent error
        # Formula: |computed - reference| / reference * 100
        error_pct = # YOUR CODE HERE
        
        print(f"{param:<10} {Q_gpa[i]:<12.3f} {Q_ref[i]:<12.3f} {error_pct:<10.3f}")
    
    # TODO: Calculate and display average absolute error
    avg_error = # YOUR CODE HERE (average of all error percentages)
    print(f"\nAverage absolute error: {avg_error:.3f}%")

# Add method to class
VFMPiecewise.compare_with_reference = compare_with_reference

### Exercise 4: Mesh Convergence Study (15 minutes)

**Theory**: VFM accuracy can depend on mesh density. We study convergence by testing multiple mesh configurations.

**Task**: Complete the mesh convergence function.

In [ ]:
def mesh_convergence_study(self, mesh_configs, Q_ref=None):
    """
    Study mesh convergence by testing multiple configurations
    
    Parameters:
    -----------
    mesh_configs : list of tuples - List of (m, n) mesh configurations
    Q_ref : array - Optional reference values for error calculation
    """
    
    print("\n" + "="*70)
    print("MESH CONVERGENCE STUDY")
    print("="*70)
    
    # TODO: Initialize storage for results
    results_list = []
    
    # TODO: Loop through each mesh configuration
    for m, n in mesh_configs:
        print(f"\nTesting {m}×{n} mesh...")
        
        # TODO: Run analysis
        Q_gpa, eta = # YOUR CODE HERE
        
        # TODO: Store results
        results_list.append({
            'm': m,
            'n': n,
            'elements': m * n,
            'Q11': Q_gpa[0],
            'Q22': Q_gpa[1],
            'Q12': Q_gpa[2],
            'Q66': Q_gpa[3]
        })
    
    # Convert to DataFrame for nice display
    df = pd.DataFrame(results_list)
    
    print("\n" + "="*70)
    print("CONVERGENCE RESULTS")
    print("="*70)
    print(df.to_string(index=False))
    
    # TODO: If reference provided, calculate errors
    if Q_ref is not None:
        print("\nErrors relative to reference (%)")
        print("-"*70)
        
        for i, row in df.iterrows():
            # TODO: Calculate errors for this mesh
            err11 = # YOUR CODE HERE
            err22 = # YOUR CODE HERE
            err12 = # YOUR CODE HERE
            err66 = # YOUR CODE HERE
            
            print(f"{row['m']}×{row['n']}: Q11={err11:.2f}%, Q22={err22:.2f}%, Q12={err12:.2f}%, Q66={err66:.2f}%")
    
    return df

# Add method to class
VFMPiecewise.mesh_convergence_study = mesh_convergence_study

### Exercise 5: Plot Strain Fields (Bonus - if time permits)

**Task**: Create contour plots of the three strain components.

In [ ]:
def plot_strain_fields(self, save_path='strain_fields.png'):
    """
    Plot the three strain field components
    """
    
    # Get 2D data
    X1 = self.data['X1_2D']
    X2 = self.data['X2_2D']
    Eps1 = self.data['Eps1_2D']
    Eps2 = self.data['Eps2_2D']
    Eps6 = self.data['Eps6_2D']
    
    # TODO: Create 1x3 subplot figure
    fig, axes = # YOUR CODE HERE (plt.subplots)
    
    # TODO: Plot ε₁
    im1 = axes[0].contourf(# YOUR CODE HERE - X1, X2, Eps1, levels, cmap)
    axes[0].set_title(r'$\varepsilon_1$', fontsize=16)
    axes[0].set_xlabel(r'$x_1$ (mm)', fontsize=14)
    axes[0].set_ylabel(r'$x_2$ (mm)', fontsize=14)
    axes[0].set_aspect('equal')
    plt.colorbar(im1, ax=axes[0])
    
    # TODO: Plot ε₂
    im2 = axes[1].contourf(# YOUR CODE HERE)
    axes[1].set_title(r'$\varepsilon_2$', fontsize=16)
    axes[1].set_xlabel(r'$x_1$ (mm)', fontsize=14)
    axes[1].set_ylabel(r'$x_2$ (mm)', fontsize=14)
    axes[1].set_aspect('equal')
    plt.colorbar(im2, ax=axes[1])
    
    # TODO: Plot ε₆
    im3 = axes[2].contourf(# YOUR CODE HERE)
    axes[2].set_title(r'$\varepsilon_6$', fontsize=16)
    axes[2].set_xlabel(r'$x_1$ (mm)', fontsize=14)
    axes[2].set_ylabel(r'$x_2$ (mm)', fontsize=14)
    axes[2].set_aspect('equal')
    plt.colorbar(im3, ax=axes[2])
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"Strain fields saved as '{save_path}'")
    plt.show()

# Add method to class
VFMPiecewise.plot_strain_fields = plot_strain_fields

---
## Running Your Analysis
---

### Step 1: Initialize and Load Data

In [ ]:
# Create VFM instance and load data
vfm = VFMPiecewise(data_source='csv')
vfm.load_data_from_csv('.')  # CSV files must be in current directory

### Step 2: Explore the Data (Exercise 1)

In [ ]:
# Display data summary
vfm.display_data_summary()

### Step 3: Run VFM Analysis (Exercise 2)

In [ ]:
# Run single analysis with 5×4 mesh
Q_gpa, eta = vfm.run_analysis(m=5, n=4)

### Step 4: Compare with Reference (Exercise 3)

In [ ]:
# Reference values (example: Wood material)
Q_ref = np.array([15.536, 1.965, 0.926, 1.109])  # Q11, Q22, Q12, Q66 in GPa

vfm.compare_with_reference(Q_ref, material_name="Wood")

### Step 5: Mesh Convergence Study (Exercise 4)

In [ ]:
# Test multiple mesh configurations
mesh_configs = [(3, 2), (5, 4), (7, 6), (10, 8)]

convergence_df = vfm.mesh_convergence_study(mesh_configs, Q_ref=Q_ref)

### Step 6: Visualize Strain Fields (Bonus Exercise 5)

In [ ]:
# Plot strain fields
vfm.plot_strain_fields()

---
## Discussion Questions

After completing your analysis, reflect on:

1. **Sensitivity Analysis**: 
   - Which parameter has the highest η/Q ratio?
   - What does this tell us about measurement sensitivity?

2. **Mesh Convergence**:
   - Do results converge as mesh density increases?
   - Is there a point where refinement doesn't help?

3. **Physical Interpretation**:
   - Are all identified parameters physically reasonable?
   - Which parameter is easiest/hardest to identify accurately?

4. **Error Sources**:
   - What could cause discrepancies with reference values?
   - How would measurement noise affect results?

5. **Material Behavior**:
   - For your material, which stiffness dominates?
   - Does Q12/Q11 ratio match material symmetry expectations?

---

## Solution Checklist

Before considering complete:

- [ ] Exercise 1: Data summary displays all key information
- [ ] Exercise 2: VFM runs and displays Q and η values correctly
- [ ] Exercise 3: Error calculations work properly
- [ ] Exercise 4: Mesh convergence study completes for all meshes
- [ ] Exercise 5: Strain field plots display correctly
- [ ] All code runs without errors
- [ ] Results are physically reasonable (positive stiffness values)
- [ ] Discussion questions answered

---

**Good luck with your VFM analysis!**